In [106]:
import google.generativeai as genai
import os
import json
from dotenv import load_dotenv
import requests
#import minsearch
from elasticsearch import Elasticsearch

In [107]:
print(genai.__version__)

0.8.1


In [108]:
es_client = Elasticsearch('http://localhost:9200') 

In [109]:
load_dotenv()
Gemini_API_KEY=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=Gemini_API_KEY)


In [110]:
#model = genai.GenerativeModel('gemini-pro')

In [111]:
with open('/workspaces/vaidya-zoomcamp24/News_Category_Dataset_v3.json') as f:
    documents=json.load(f)
    print(documents[0])

{'link': 'https://www.huffpost.com/entry/covid-boosters-uptake-us_n_632d719ee4b087fae6feaac9', 'headline': 'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters', 'category': 'U.S. NEWS', 'short_description': 'Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.', 'authors': 'Carla K. Johnson, AP', 'date': '2022-09-23'}


In [112]:
"""index=minsearch.Index(text_fields=["headline", "short_description"],
    keyword_fields=["category"]
)
index.fit(documents)"""

'index=minsearch.Index(text_fields=["headline", "short_description"],\n    keyword_fields=["category"]\n)\nindex.fit(documents)'

In [113]:
documents[0]["headline"]

'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters'

In [114]:
from tqdm.auto import tqdm

In [115]:
for doc in tqdm(documents):
    es_client.index(index="news_index", document=doc)

  0%|          | 0/569 [00:00<?, ?it/s]

In [134]:
es_client

<Elasticsearch(['http://localhost:9200'])>

In [116]:
def rag(query):
    results=elastic_search(query)
    #prompt=build_prompt(query,results)
    #answer=llm(prompt)
    return results#answer.text


In [117]:
"""def search(query):
    results = index.search(
        query=query,
        filter_dict={'category':'WORLD NEWS'},
        num_results=50
    )

    return results"""

"def search(query):\n    results = index.search(\n        query=query,\n        filter_dict={'category':'WORLD NEWS'},\n        num_results=50\n    )\n\n    return results"

In [129]:
def elastic_search(query):
    """search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["headline", "short_description"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "category": "COMEDY"
                    }
                }
            }
        }
    }"""
search_query = {
                "query": {
                "match_all": {}
                        }
                }

response = es_client.search(index="news_index", body=search_query, size=10000)


/tmp/ipykernel_3894/3705805191.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index="news_index", body=search_query, size=10000)


In [130]:

def build_prompt(query,results):

    # Define the context and query
    #context = [item['short_description'] for item in results]
    context = ""
    
    for doc in results:
        context = context + f"section: {doc['headline']}\n  question: {doc['short_description']}"
    
    
    # Define the prompt template
    prompt_template =  f""" {query} ? .Answer from the section which is delimited with triple backticks?Review text:
                        '''{context}'''
                        """.strip()
    
    return prompt_template
    
def llm(prompt):
# Initialize the model
    model = genai.GenerativeModel(model_name='gemini-pro')
    
    # Start a chat session
    chat = model.start_chat(history=[])
    
    # Send the message with function calling enabled within the chat session
    gemini_response = chat.send_message(prompt)
    
    #gemini_response = response.text
    return gemini_response



In [131]:
query="American Airlines"
answer=rag(query)
print(answer)

None
